## Deliverable 2. Create a Customer Travel Destinations Map.

In [13]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [15]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Sioux Lookout,CA,50.1001,-91.9170,-1.82,65,20,3.44,few clouds
1,1,Avarua,CK,-21.2078,-159.7750,82.45,83,20,17.27,few clouds
2,2,Ushuaia,AR,-54.8000,-68.3000,51.42,76,75,16.11,broken clouds
3,3,Hovd,MN,48.0056,91.6419,-4.00,92,100,2.33,overcast clouds
4,4,Aklavik,CA,68.2191,-135.0107,-9.40,64,75,5.75,broken clouds


In [16]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [17]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]

preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Avarua,CK,-21.2078,-159.7750,82.45,83,20,17.27,few clouds
9,9,Bambous Virieux,MU,-20.3428,57.7575,75.45,83,40,3.44,scattered clouds
11,11,Kencong,ID,-8.2833,113.3667,76.33,92,100,1.79,overcast clouds
13,13,Labuhan,ID,-6.8844,112.2051,77.99,86,100,9.89,light rain
15,15,Biak,ID,-0.9131,122.8766,76.32,85,99,1.50,overcast clouds
17,17,Rikitea,PF,-23.1203,-134.9692,78.78,76,94,17.49,overcast clouds
24,24,Hobart,AU,-42.8794,147.3294,82.26,61,0,1.01,clear sky
25,25,Vaini,TO,-21.2000,-175.2000,82.81,77,92,15.64,overcast clouds
27,27,Makakilo City,US,21.3469,-158.0858,80.92,50,0,9.22,clear sky
28,28,Souillac,MU,-20.5167,57.5167,75.58,83,40,3.44,scattered clouds


In [18]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

Unnamed: 0             0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [19]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [20]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Avarua,CK,82.45,few clouds,-21.2078,-159.7750,
9,Bambous Virieux,MU,75.45,scattered clouds,-20.3428,57.7575,
11,Kencong,ID,76.33,overcast clouds,-8.2833,113.3667,
13,Labuhan,ID,77.99,light rain,-6.8844,112.2051,
15,Biak,ID,76.32,overcast clouds,-0.9131,122.8766,
17,Rikitea,PF,78.78,overcast clouds,-23.1203,-134.9692,
24,Hobart,AU,82.26,clear sky,-42.8794,147.3294,
25,Vaini,TO,82.81,overcast clouds,-21.2000,-175.2000,
27,Makakilo City,US,80.92,clear sky,21.3469,-158.0858,
28,Souillac,MU,75.58,scattered clouds,-20.5167,57.5167,


In [21]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()

    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [22]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df.tail(30)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
541,Maxixe,MZ,75.31,broken clouds,-23.8597,35.3472,Oceano
554,Mackay,AU,82.38,broken clouds,-21.1500,149.2000,International Lodge Motel
562,Pandan,PH,80.62,broken clouds,14.0453,124.1694,Bonifacio Lodge
566,Axim,GH,78.75,few clouds,4.8699,-2.2405,Axim Beach Resort And Spa
568,Fare,PF,80.44,light rain,-16.7000,-151.0167,Maitai Lapita Village Huahine
577,Oistins,BB,75.29,few clouds,13.0667,-59.5333,Butterfly Beach Hotel
583,Jales,BR,79.21,overcast clouds,-20.2689,-50.5458,Geraldelli Center Hotel
588,Tambilil,PH,83.64,scattered clouds,5.9622,124.6697,Wilfredo Sotto Home
589,Kulhudhuffushi,MV,79.12,scattered clouds,6.6221,73.0700,Haajy Guest House 1
590,Conde,BR,76.46,moderate rain,-7.2597,-34.9075,Pousada Beija Flor


In [23]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [24]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [25]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))